# Submission Visualization

I simplified the visualization codes from @jpbremer and @corochann

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import numpy as np
import pandas as pd

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

import os

from matplotlib import animation, rc
from IPython.display import HTML, display

rc('animation', html='jshtml')

def set_seed(seed):
    # random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)

In [ ]:
set_seed(42)

In [ ]:
# set env variable for data
data_path = '../input/lyft-motion-prediction-autonomous-vehicles'
os.environ["L5KIT_DATA_FOLDER"] = data_path
# get config
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")
cfg['model_params']['history_num_frames'] = 10  # note when training model, we set this to 10
cfg['raster_params']['disable_traffic_light_faces'] = False
cfg['test_data_loader'] = {
    'key': 'scenes/test.zarr',
    'batch_size': 128,
    'shuffle': False,
    'num_workers': 4,
}
print(cfg)

# Load the test data

In [ ]:
dm = LocalDataManager()
zarr_dataset = ChunkedDataset(dm.require(cfg['test_data_loader']['key'])).open()
print(zarr_dataset)

# Build the Rasterizers

In [ ]:
# Semantic view
cfg['raster_params']['map_type'] = 'py_semantic'
semantic_rasterizer = build_rasterizer(cfg, dm)

# Satellite view
cfg['raster_params']['map_type'] = 'py_satellite'
satellite_rasterizer = build_rasterizer(cfg, dm)

# Read submission.csv
Add output from your submission notebook as data on the right. Then enter the path to the csv below

In [ ]:
df_sub = pd.read_csv('../input/lyft-complete-train-and-prediction-pipeline/submission.csv')
df_sub = df_sub.set_index(['timestamp', 'track_id'])

In [ ]:
display(df_sub)

In [ ]:
def row_to_confs(row):
    return [row[f'conf_{i}'] for i in range(3)]
def row_to_coords(row):
    return row[3:].values.reshape(3, 50, 2)

# Frame

## Autonomous Vehicle (Ego)

In [ ]:
semantic_dataset = EgoDataset(cfg, zarr_dataset, semantic_rasterizer)
satellite_dataset = EgoDataset(cfg, zarr_dataset, satellite_rasterizer)
test_mask = np.load(f"{data_path}/scenes/mask.npz")["arr_0"]
agent_semantic_dataset = AgentDataset(cfg, zarr_dataset, semantic_rasterizer, agents_mask=test_mask)
agent_satellite_dataset = AgentDataset(cfg, zarr_dataset, satellite_rasterizer, agents_mask=test_mask)

In [ ]:
# here I use matplotlib default colors
cmap = plt.get_cmap("tab10")
matplotlib_colors_in_rgb_int = [
    [int(255 * x) for x in cmap(i)[:3]] for i in range(10)
]

In [ ]:
# note raster_from_agent is actually a constant matrix for each raster once you fix the raster params
raster_params = cfg['raster_params']
raster_from_agent = np.array([
    [2., 0.,  56.],
    [0., 2., 112.],
    [0., 0.,   1.],
]) if (
    raster_params['raster_size'] == [224, 224] and
    raster_params['pixel_size'] == [0.5, 0.5] and
    raster_params['ego_center'] == [0.25, 0.5]
) else None
    
def generate_image_trajectory(dataset, index):
    data = dataset[index]
    im = data['image'].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    rfg = raster_from_agent if raster_from_agent is not None else data['raster_from_agent']
    target_positions_pixels = transform_points(data['target_positions'], rfg)
    draw_trajectory(im, target_positions_pixels, yaws=data['target_yaws'], rgb_color=TARGET_POINTS_COLOR)
    return im

def plot_trajectory(dataset, indices, width=12, height=4, n_cols=3, title=''):
    if not isinstance(indices, (list, np.ndarray)):
        indices = [indices]
    n_rows = len(indices) // n_cols + len(indices) % n_cols
    plt.figure(figsize=(width, height*n_rows))
    for k, index in enumerate(indices):
        plt.subplot(n_rows, n_cols, 1+k).set_title(str(index))
        im = generate_image_trajectory(dataset, index)
        plt.imshow(im, origin='lower')
    if title:
        plt.suptitle(title)
    plt.show()

def generate_image_predicted_trajectory(dataset, df_sub, index):
    data = dataset[index]
    im = data['image'].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    row = df_sub.loc[(data['timestamp'], data['track_id'])]
    # note submission coordinate system = world - centroid
    predicted_target_positions_in_sub = row_to_coords(row)
    predicted_target_positions_in_world = predicted_target_positions_in_sub + data['centroid']
    for i, coords in enumerate(predicted_target_positions_in_world):
        target_positions_pixels = transform_points(coords, data['raster_from_world'])
        draw_trajectory(im, target_positions_pixels, rgb_color=matplotlib_colors_in_rgb_int[i])
    return im, row_to_confs(row)

def plot_predicted_trajectory(dataset, df_sub, indices, width=12, height=4, n_cols=3, title=''):
    if not isinstance(indices, (list, np.ndarray)):
        indices = [indices]
    n_rows = len(indices) // n_cols + len(indices) % n_cols
    plt.figure(figsize=(width, height*n_rows))
    for k, index in enumerate(indices):
        plt.subplot(n_rows, n_cols, 1+k).set_title(str(index))
        im, confs = generate_image_predicted_trajectory(dataset, df_sub, index)
        patches = [mpatches.Patch(color=cmap(m), label='%.3f'%conf) for m, conf in enumerate(confs)]
        plt.imshow(im, origin='lower')
        plt.legend(handles=patches)
    if title:
        plt.suptitle(title)
    plt.show()

3 curves corresponding to 3 mode of predictions. The legend indicates the confidence scores. The bright green is history.

In [ ]:
plot_predicted_trajectory(agent_semantic_dataset, df_sub, [18431], width=6, height=6, n_cols=1)

In [ ]:
i_plots = np.random.randint(len(agent_semantic_dataset), size=9)
# i_plots = (i_plots - i_plots[0] + 18552) % len(agent_semantic_dataset)
i_plots

In [ ]:
plot_predicted_trajectory(agent_semantic_dataset, df_sub, i_plots)

In [ ]:
plot_predicted_trajectory(agent_satellite_dataset, df_sub, i_plots)

In [ ]:
# note in the test set the each agent only have very few frames.
plot_predicted_trajectory(agent_semantic_dataset, df_sub, list(range(5311, 5311+9)))

In [ ]:
# weird case
plot_predicted_trajectory(agent_semantic_dataset, df_sub, [25658], width=6, height=6, n_cols=1)

In [ ]:
# weird case
plot_predicted_trajectory(agent_satellite_dataset, df_sub, [25658], width=6, height=6, n_cols=1)

#  Entire Scene
A scene is just a lot of frames

In [ ]:
def animate(images):
    fig = plt.figure()
    ims = [(plt.imshow(im, animated=True, origin='lower'),) for im in images]
    anim = animation.ArtistAnimation(fig, ims, interval=60, blit=True, repeat_delay=1000)
    plt.close()
    return HTML(anim.to_jshtml())

def plot_scene(dataset, scene_id):
    indices = dataset.get_scene_indices(scene_id)
    print('scene', scene_id, ':', indices[0], '-', indices[-1])
    images = [generate_image_trajectory(dataset, i) for i in indices]
    return animate(images)

import bisect
def get_scene_index_from_frame_id(dataset, frame_id):
    return bisect.bisect_right(dataset.cumulative_sizes, frame_id)

In [ ]:
scene_id = get_scene_index_from_frame_id(semantic_dataset, i_plots[0])

In [ ]:
plot_scene(semantic_dataset, scene_id)

In [ ]:
plot_scene(satellite_dataset, scene_id)

### Agent
Note one agent scene contains lots of agents. So you will see the image center jump around at each agent

In [ ]:
agent_scene_id = get_scene_index_from_frame_id(agent_semantic_dataset, i_plots[4])

In [ ]:
plot_scene(agent_semantic_dataset, agent_scene_id)

In [ ]:
plot_scene(agent_satellite_dataset, agent_scene_id)

In [ ]:
# see what each agent were doing
plot_trajectory(agent_semantic_dataset, list(range(47930, 47930+9)))

### References
* https://www.kaggle.com/corochann/lyft-comprehensive-guide-to-start-competition/
* https://www.kaggle.com/jpbremer/lyft-scene-visualisations